# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3 as sq
import re
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import f1_score

In [4]:
# load cleaned data from database
df = pd.read_sql("SELECT * FROM messages_and_categories", sq.connect("/content/drive/MyDrive/datasets/ds-data-engineering/Disaster_Messages.db"))

X = df["message"]
Y = df.drop(["message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    PARAMETERS:
    text - raw string

    RETURN:
    tokens - list of token word
    '''
    # make all lowercase and delete punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    #tokenize and lemmatize
    tokens = word_tokenize(text.lower())
    tokens = [WordNetLemmatizer().lemmatize(token) for token in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

Create functions for load and save the model to file

In [7]:
def load_or_fit(pipeline, file_dir, x, y):
  '''
  DESCRIPTION:
  Check if file directory exist. if exist, load the file to model.\
  if not, train the pipeline

  PARAMETERS:
  pipeline (Pipeline) - Pipeline Object
  file_dir (str) - file directory
  x (DataFrame) - X value for training
  y (DataFrame) - Y value for training

  RETURN:
  pipeline (Pipeline) - Trained pipeline
  '''

  try:
    file = open(file_dir)
    print('Load from file')
    pipeline = pickle.load(open(file_dir, "rb"))
  except:
    print('Train the model')
    pipeline.fit(x,y)
  
  return pipeline

In [8]:
def save_model(obj, file_dir):
  '''
  PARAMETERS:
  pipeline (Pipeline) - Pipeline Object
  file_dir (str) - file directory
  
  '''
  with open(file_dir, 'wb') as dir:
    pickle.dump(obj, dir)

split into train and test dataset

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)

train or load the model

In [10]:
model1_dir = r'/content/drive/MyDrive/datasets/ds-data-engineering/model1.pickle'
pipeline = load_or_fit(pipeline, model1_dir, x_train, y_train)

Train the model


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
#convert numpy array to DataFrame
y_preds = pipeline.predict(x_test)
y_preds = pd.DataFrame(y_preds, columns=y_test.columns)

create function to calculate the f1_score mean of all columns

In [12]:
def mean_f1(truth, preds):
  '''
  PARAMETERS:
  preds (DataFrame) - DataFrame from prediction
  truth (DataFrame) - Truth DataFrame

  RETURN:
  mean (float) - mean of all f1_score from all columns
  '''
  f1_score_list = []
  for col_name in y_preds.columns:
    f1_score_list.append(f1_score(truth[col_name], preds[col_name], average='weighted'))
  
  return np.mean(f1_score_list)

In [13]:
mean_f1(y_test, y_preds)

0.9308773990395723

In [14]:
save_model(pipeline, model1_dir)

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
    'clf__estimator__n_estimators': [50,100],
    'clf__estimator__min_samples_split': [2, 4],
    'tfidf__use_idf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [16]:
model2_dir = r'/content/drive/MyDrive/datasets/ds-data-engineering/model2.pickle'
cv = load_or_fit(cv, model2_dir, x_train, y_train)

Train the model


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
#convert numpy array to DataFrame
y_preds2 = cv.predict(x_test)
y_preds2 = pd.DataFrame(y_preds, columns=y_test.columns)

In [18]:
mean_f1(y_test, y_preds2)

0.9308773990395723

In [19]:
#save the GirdSearch
save_model(cv, model2_dir)

#save the best model
best_model2_dir = r'/content/drive/MyDrive/datasets/ds-data-engineering/best_model2.pickle'
save_model(cv.best_estimator_, best_model2_dir)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Create another feature

In [20]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

  def starting_verb(self, text):
    sentence_list = nltk.sent_tokenize(text)
    for sentence in sentence_list:
      pos_tags = nltk.pos_tag(tokenize(sentence))
      if len(pos_tags) != 0:
        first_word, first_tag = pos_tags[0]
      else:
        first_word, first_tag = (' ', ' ')
      if first_tag in ['VB', 'VBP'] or first_word == 'RT':
        return True
    return False

  def fit(self, x, y=None):
    return self

  def transform(self, X):
    X_tagged = pd.Series(X).apply(self.starting_verb)
    return pd.DataFrame(X_tagged)

In [21]:
pipeline2 = Pipeline([
  ('feature_union', FeatureUnion([
    ('text_pipeline', Pipeline([
      ('vect', CountVectorizer(tokenizer=tokenize)),
      ('tfidf', TfidfTransformer())
    ])),
    ('verb_extractor', StartingVerbExtractor())                   
  ])),
  ('clf', MultiOutputClassifier(ExtraTreesClassifier()))
])

In [22]:
model3_dir = r'/content/drive/MyDrive/datasets/ds-data-engineering/model3.pickle'
pipeline2 = load_or_fit(pipeline2, model3_dir, x_train, y_train)

Train the model


In [23]:
#convert numpy array to DataFrame
y_preds3 = pd.DataFrame(pipeline2.predict(x_test), columns=y_test.columns)

In [24]:
mean_f1(y_test, y_preds3)

0.9288810605150856

### 9. Export your model as a pickle file

In [25]:
save_model(pipeline2, model3_dir)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

###11. Conclusion
Result:
- model 1 f1_score = 0.93087
- model 2 f1_score = 0.93087
- model 3 f1_score = 0.92888

since the models with highest f1 score are model 1 and model 2, so we will use model 1 for the backend of the web app.


you can download the model [here](https://drive.google.com/drive/folders/1010BLJ5g3fhpq25OZ08QJPSMHjiInJ3t?usp=sharing)